In [4]:
import requests
from io import BytesIO
from PIL import Image, ImageFilter
import numpy as np

class ImageURLExtractor(object):
    
    #default image size and dimension
    img_rows, img_cols, dim = 100, 100, 1
    
    def __init__(self, information):
        self.information = information
    
    def X_train_data(self, inputTransform):
        no_of_images = len(self.information.trainingDataList)
        data = np.random.random((no_of_images, self.dim, inputTransform.transformSize[0].dimensionSize, inputTransform.transformSize[1].dimensionSize))
        image_index = 0
        
        if self.information.extractor == type(self).__name__:
            for trainingData in self.information.trainingDataList:
                response = requests.get(trainingData.URL)
                img = Image.open(BytesIO(response.content))
                
                #FIXME: move transformer to transform package
                img_rows = inputTransform.transformSize[0].dimensionSize
                img_cols = inputTransform.transformSize[1].dimensionSize
                img = img.resize((img_rows,img_cols), Image.ANTIALIAS)
                
                for parameter in inputTransform.transformParam:
                    if parameter.parameterName == 'color':
                        if parameter.parameterValue == 'grey':
                            img = img.convert("L")         
                        if parameter.parameterName == 'process':
                            if parameter.parameterValue == 'edge':
                                img = img.filter(ImageFilter.FIND_EDGES)
                            
                img = np.asarray(img, dtype=np.float32)     
                data[image_index, 0, :, :] = img
                print (img.shape)
                print (trainingData.URL)
                
        else:
            print ("Extractor in the information file and running extractor is not matching.")
        return data
    
    def Y_train_data(self, inputTransform):
        print (self.information.extractor)
        return
    
    def X_action_data(self, inputTransform):
        print (self.information.extractor)
        return   

import scientists_pb2
import sys
import personaDefinition_pb2

information = scientists_pb2.Information()

persona = personaDefinition_pb2.Persona()
dna = persona.DNAs.add()
inputLayer = dna.inputs.add()
inputLayer.inputTransform.transformerName = "imageTransform"
inputLayer.inputTransform.informationType = "image"
transformInputSize1 =  inputLayer.inputTransform.transformSize.add()
transformInputSize1.dimension = 1
transformInputSize1.dimensionSize = 50
transformInputSize2 =  inputLayer.inputTransform.transformSize.add()
transformInputSize2.dimension = 1
transformInputSize2.dimensionSize = 50
transformParam1 = inputLayer.inputTransform.transformParam.add()
transformParam1.parameterName = "color"
transformParam1.parameterValue  = "grey"
transformParam2 = inputLayer.inputTransform.transformParam.add()
transformParam2.parameterName = "process"
transformParam2.parameterValue  = "edge"
    
# Read the existing address book.
f = open("test.bin", "rb")
information.ParseFromString(f.read())
f.close()

imageURLExtractor = ImageURLExtractor(information)
imageURLExtractor.X_train_data(inputLayer.inputTransform)

(50L, 50L)
http://www.loc.gov/rr/scitech/subjectguides/images/tesla-new.jpg
(50L, 50L)
https://static1.squarespace.com/static/51cdd10de4b08819bd7bc9b4/525d89c2e4b0f8245cabfc96/53cfd660e4b089801fe7e3d9/1458152617516/SciSource_BK0929.jpg?format=750w
(50L, 50L)
https://static1.squarespace.com/static/51cdd10de4b08819bd7bc9b4/525d89c2e4b0f8245cabfc96/53cfd28be4b0c47638e2f540/1458152692173/SciSource_BN4337.jpg?format=750w


array([[[[  1.13000000e+02,   1.14000000e+02,   1.15000000e+02, ...,
            1.13000000e+02,   1.12000000e+02,   1.12000000e+02],
         [  1.13000000e+02,   1.13000000e+02,   1.13000000e+02, ...,
            1.13000000e+02,   1.11000000e+02,   1.12000000e+02],
         [  1.13000000e+02,   1.12000000e+02,   1.13000000e+02, ...,
            1.12000000e+02,   1.12000000e+02,   1.13000000e+02],
         ..., 
         [  3.40000000e+01,   3.30000000e+01,   3.00000000e+01, ...,
            4.20000000e+01,   4.10000000e+01,   4.20000000e+01],
         [  3.60000000e+01,   3.60000000e+01,   3.50000000e+01, ...,
            4.10000000e+01,   4.10000000e+01,   4.10000000e+01],
         [  3.70000000e+01,   3.70000000e+01,   3.70000000e+01, ...,
            4.10000000e+01,   4.10000000e+01,   4.00000000e+01]]],


       [[[  8.89001366e-01,   9.58417676e-01,   6.70586307e-01, ...,
            9.80330850e-02,   7.32524329e-01,   9.58235752e-01],
         [  2.45190641e-01,   2.40903362e-0